In [63]:
import os


In [64]:
%pwd

'c:\\'

In [65]:
os.chdir("../")

In [66]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    alpha: float
    target_column: str

In [67]:
from mlflowProject.constants import *
from mlflowProject.utils.common import read_yaml,create_directories

In [68]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Lasso
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        target_column = list(schema.keys())[0] if schema else "OutletSales"

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            target_column = target_column
        )

        return model_trainer_config


In [69]:
import pandas as pd
import numpy as np
import os
from mlflowProject import logger
from sklearn.linear_model import Lasso
import joblib

In [70]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        train_x=pd.get_dummies(train_x)
        train_y=np.log(train_y)

        model1 = Lasso(alpha=self.config.alpha,random_state=42,max_iter=1000,tol=0.1)
        model1.fit(train_x, train_y)

        joblib.dump(model1, os.path.join(self.config.root_dir, self.config.model_name))

In [71]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
    
except Exception as e:
    raise e

[2024-11-22 09:55:33,704: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\config\config.yaml]
[2024-11-22 09:55:33,709: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\params.yaml]
[2024-11-22 09:55:33,716: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\schema.yaml]
[2024-11-22 09:55:33,720: INFO: common: created directory at artifacts]
[2024-11-22 09:55:33,723: INFO: common: created directory at artifacts/model_trainer]


In [72]:
data = pd.read_csv(r"C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\artifacts\data_transformation\train.csv")
data['OutletSales'].dtypes

dtype('float64')

In [73]:
data.isnull().sum()

ProductID            0
Weight               0
FatContent           0
ProductVisibility    0
ProductType          0
MRP                  0
OutletID             0
EstablishmentYear    0
OutletSize           0
LocationType         0
OutletType           0
OutletSales          0
dtype: int64